In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time

In [4]:

# Configuración de la URL que queremos scrappear y los encabezados HTTP

BASE_URL = "https://co.computrabajo.com/trabajo-de-analista-de-datos"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

respuesta = requests.get(BASE_URL, headers=HEADERS)

In [5]:
# Creamos una lista vacia donde vamos a añadir las ofertas de trabajo
data = []

In [ ]:
# Número máximo de páginas que queremos scrappear
max_paginas = int(input("Cuántas páginas deseas scrappear?: "))

pagina = 1

while pagina <= max_paginas:
    # Construimos la url
    url_pagina = BASE_URL if pagina == 1 else f"{BASE_URL}?p={pagina}"
    print(f"Procesando página {pagina} - {url_pagina}")


    try:
        respuesta = requests.get(url_pagina, headers=HEADERS)
        respuesta.raise_for_status()  # controlamos la respuesta HTTP en caso de que algun error
    except requests.exceptions.RequestException as e:
        print(f"Error al hacer la solicitud: {e}")
        break

    if respuesta.status_code == 200:
        soup = BeautifulSoup(respuesta.text, "html.parser")
        
        # Encontrar todas las ofertas de trabajo
        ofertas = soup.find_all("article", class_="box_offer")
        
        # si no hay mas ofertas, detenemos el bucle
        if not ofertas:
            print("No se encontraron más ofertas")
            break

        for oferta in ofertas:
            titulo_elemento = oferta.find("a", class_="js-o-link fc_base")
            empresa_elemento = oferta.find("a", class_="fc_base t_ellipsis")
            ubicacion_elemento = oferta.find("p", class_="fs16 fc_base mt5")  
            salario_elemento = oferta.find("span", class_="dIB mr10")

            # Extraer texto con None
            titulo = titulo_elemento.get_text(strip=True) if titulo_elemento else "No disponible"
            enlace = f"https://co.computrabajo.com{titulo_elemento['href']}" if titulo_elemento else "No disponible"
            empresa = empresa_elemento.get_text(strip=True) if empresa_elemento else "No disponible"
            ubicacion = ubicacion_elemento.get_text(strip=True) if ubicacion_elemento else "No disponible"
            salario = salario_elemento.get_text(strip=True) if salario_elemento else "No disponible"

            # Agregamos los datos a la lista vacia
            data.append({
                "Título": titulo,
                "Empresa": empresa,
                "Ubicación": ubicacion,
                "Salario": salario,
                "Enlace": enlace
            })

        # Pausa aleatoria para evitar bloqueos, entre 1 y 3 segundos, puede ser mas para que estemos mas seguros
        time.sleep(random.uniform(1, 3))

        # Pasamos a la siguiente pagina
        pagina += 1
    else:
        print(f"Error al acceder a la página, estado: {respuesta.status_code}")
        break


In [ ]:
df = pd.DataFrame(data) # creamos un Dataframe con la data que obtuvimos
df

In [8]:

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Función para extraer la descripción de una oferta individual
def obtener_descripcion(enlace):
    try:
        respuesta = requests.get(enlace, headers=HEADERS)
        respuesta.raise_for_status()
        soup = BeautifulSoup(respuesta.text, "html.parser")
        descripcion_elemento = soup.find("p", class_="mbB")
        
        return descripcion_elemento.get_text(strip=True) if descripcion_elemento else "No disponible"
    
    except requests.exceptions.RequestException as e:
        print(f"Error al acceder al enlace {enlace}: {e}")
        return "Error"


# Obtenemos las descripciones
descripciones = []
for enlace in df["Enlace"]:
    descripcion = obtener_descripcion(enlace)
    descripciones.append(descripcion)
    time.sleep(random.uniform(1, 3))

# luego creamos una nueva columna a la cual le vamos a añadir las descripciones
df["Descripción"] = descripciones


In [ ]:
df

In [12]:
df.to_csv("Ofertas_Computrabajo.csv",index=False, encoding="utf-8-sig")